In [1]:
# !pip install requests python-dotenv pandas

from dotenv import load_dotenv
import os
load_dotenv()  # will auto-detect .env in current working directory

print("Has key:", bool(os.getenv("NEWSAPI_KEY")))

import os, requests, pandas as pd
from dotenv import load_dotenv
load_dotenv()

NEWSAPI_KEY = os.getenv("NEWSAPI_KEY")

def get_news(query="AAPL", limit=10):
    if not NEWSAPI_KEY:
        raise RuntimeError("Missing NEWSAPI_KEY in environment (.env).")

    url = "https://newsapi.org/v2/everything"
    params = {
        "q": query,
        "sortBy": "publishedAt",
        "language": "en",
        "pageSize": limit,
        "apiKey": NEWSAPI_KEY
    }
    r = requests.get(url, params=params, timeout=15)
    try:
        data = r.json()
    except Exception as e:
        raise RuntimeError(f"Non-JSON response ({r.status_code}): {r.text[:200]}") from e

    # Explicitly surface API errors (e.g., key missing, rate limited)
    if data.get("status") != "ok":
        msg = data.get("message", "Unknown NewsAPI error")
        code = data.get("code", "no-code")
        raise RuntimeError(f"NewsAPI error: {code} — {msg}")

    articles = data.get("articles", [])
    df = pd.DataFrame([{
        "source": a["source"]["name"],
        "title": a["title"],
        "description": a["description"],
        "publishedAt": a["publishedAt"],
        "url": a["url"]
    } for a in articles])

    if df.empty:
        print(f"No articles found for query '{query}'.")
    return df

# demo
df = get_news("Tesla", limit=5)
df


Has key: True


,source,title,description,publishedAt,url
0,Yahoo Entertainment,Tesla finally resolves controversial design fl...,"""Maybe we deleted too much.""",2025-10-26T00:45:00Z,https://autos.yahoo.com/ev-and-future-tech/art...
1,Yahoo Entertainment,Plynk Launches $1 Minimum Investment App Featu...,Digital investing app Plynk on Thursday launch...,2025-10-26T00:08:54Z,https://finance.yahoo.com/news/plynk-launches-...
2,Seclists.org,(no subject),Posted by RISKS List Owner on Oct 25Content-Ty...,2025-10-25T23:49:13Z,https://seclists.org/risks/2025/q4/2
3,New Zealand Herald,Inside the rise of private concierge services ...,New York Times: Concierge services vary by siz...,2025-10-25T23:00:00Z,https://www.nzherald.co.nz/business/personal-f...
4,Wnd.com,Gas-powered cars would have crushed EVs in the...,"'Elon Musk is brilliant, but he also has fortu...",2025-10-25T22:53:55Z,https://www.wnd.com/2025/10/gas-powered-cars-w...
